# **Introdução**

## Ajuste e previsão: fundamentos do estimador<br>
O Scikit-learn fornece dezenas de algoritmos e modelos de aprendizado de máquina integrados, chamados estimadores . Cada estimador pode ser ajustado a alguns dados usando seu método de ajuste .

Abaixo está um exemplo simples onde ajustamos um ***RandomForestClassifiera*** a alguns dados muito básicos.

Uma floresta aleatória é um meta estimador que ajusta vários classificadores de árvore de decisão em várias subamostras do conjunto de dados e usa a média para melhorar a precisão preditiva e controlar o ajuste excessivo. O tamanho da subamostra é controlado com o max_samplesparâmetro if bootstrap=True(padrão), caso contrário, todo o conjunto de dados é usado para construir cada árvore.

In [1]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
X = [[ 1,  2,  3],  # 2 amostras, 3 features
     [11, 12, 13]]
y = [0, 1]          # classes de cada amostra
clf.fit(X, y)

RandomForestClassifier(random_state=0)

O método de ajuste geralmente aceita 2 entradas:

* A matriz de amostras (ou matriz de projeto) X . O tamanho de X normalmente é , o que significa que as amostras são representadas como linhas e os recursos são representados como colunas.(n_samples, n_features)

* Os valores alvo y que são números reais para tarefas de regressão ou inteiros para classificação (ou qualquer outro conjunto discreto de valores). Para tarefas de aprendizado não supervisionadas, ynão precisa ser especificado. yé geralmente 1d array onde a iª entrada corresponde ao destino da iª amostra (linha) de X.

Ambos Xe ygeralmente são matrizes numpy ou tipos de dados semelhantes a matrizes equivalentes , embora alguns estimadores trabalhem com outros formatos, como matrizes esparsas.

Uma vez que o estimador esteja ajustado, ele pode ser usado para prever valores alvo de novos dados. Você não precisa treinar novamente o estimador:

In [2]:
clf.predict(X)                          # Prevê classes dos dados de treinamento
clf.predict([[4, 5, 6], [14, 15, 16]])  # prevê classes de novos dados

array([0, 1])

## Transformadores e pré-processadores 
Os fluxos de trabalho de aprendizado de máquina geralmente são compostos de diferentes partes. Um pipeline típico consiste em uma etapa de pré-processamento que transforma ou introduz os dados e um preditor final que prevê valores de destino.

No scikit-learn, pré-processadores e transformadores seguem a mesma API que os objetos estimadores (na verdade, todos eles herdam da mesma classe BaseEstimator). Os objetos do transformador não têm um método de previsão , mas sim um método de transformação que gera uma matriz de amostra recém-transformada X:

In [4]:
from sklearn.preprocessing import StandardScaler
X = [[0, 15],
     [1, -10]]
# scale data according to computed scaling values
StandardScaler().fit(X).transform(X)

array([[-1.,  1.],
       [ 1., -1.]])

Às vezes, você deseja aplicar diferentes transformações a diferentes recursos: o ColumnTransformer é projetado para esses casos de uso.

## Pipelines: encadeamento de pré-processadores e estimadores

Transformadores e estimadores (preditores) podem ser combinados em um único objeto unificador: um Pipeline. O pipeline oferece a mesma API que um estimador regular: ele pode ser ajustado e usado para previsão com *fit* e *predict*. Como veremos mais adiante, usar um pipeline também evitará vazamento de dados, ou seja, divulgar alguns dados de teste em seus dados de treinamento.

No exemplo a seguir, carregamos o conjunto de dados ***Iris*** , dividimos em conjuntos de treinamento e de teste e calculamos a pontuação de precisão de um pipeline nos dados de teste:

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cria um objeto pipeline
pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression()
)

#carrega uma base de dados. Neste exemplo, a base de dados "Iris"
X, y = load_iris(return_X_y=True)

#divide a base de dados em conjuntos de treinamento e de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Ajusta o pipeline como um todo
pipe.fit(X_train, y_train)

#podemos agora usá-lo como outro estimador qualquer
accuracy_score(pipe.predict(X_test), y_test) #cálculo da pontuação de previsão

0.9736842105263158

## Avaliação do modelo

Ajustar um modelo a alguns dados não implica que ele preveja bem dados não vistos. Isso precisa ser avaliado diretamente. Acabamos de ver o ***train_test_split*** auxiliar que divide um conjunto de dados em conjuntos de **treinamento** e de **teste**, mas scikit-learn fornece muitas outras ferramentas para avaliação de modelos, em particular para **validação cruzada** .

Aqui mostramos brevemente como realizar um procedimento de validação cruzada de 5 vezes, usando o ***cross_validateauxiliar***. Observe que também é possível iterar manualmente nas dobras, usar diferentes estratégias de divisão de dados e usar funções de pontuação personalizadas. Consulte o [Guia do usuário do Scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) para obter mais detalhes.

In [8]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

X, y = make_regression(n_samples=1000, random_state=0)
rl = LinearRegression()

resultado = cross_validate(rl, X, y)    # por padrão, faz uma validação cruzada de 5 vezes
resultado['test_score']                 #a pontuação r_quadrado é alta porque a base de dados é simples

array([1., 1., 1., 1., 1.])

## Pesquisas automáticas de parâmetros 

Todos os estimadores possuem parâmetros (muitas vezes chamados de **hiperparâmetros** na literatura) que podem ser ajustados. O poder de generalização de um estimador muitas vezes depende criticamente de alguns parâmetros. Por exemplo, a ***RandomForestRegressor*** tem um parâmetro ***n_estimators*** que determina o número de árvores na floresta e um parâmetro ***max_depth*** que determina a profundidade máxima de cada árvore. Muitas vezes, não está claro quais devem ser os valores exatos desses parâmetros, pois eles dependem dos dados disponíveis.

O Scikit-learn fornece ferramentas para encontrar automaticamente as melhores combinações de parâmetros (via validação cruzada). No exemplo a seguir, pesquisamos aleatoriamente no espaço de parâmetros de uma floresta aleatória com um objeto ***RandomizedSearchCV***. Quando a pesquisa termina, o ***RandomizedSearchCV*** se comporta como um ***RandomForestRegressor*** que foi ajustado com o melhor conjunto de parâmetros. Leia mais no [Guia do usuário do Scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation).

In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import randint

X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#definie o espaço de parâmetros que será pesquisado
param_distributions = {'n_estimators': randint(1, 5),
                       'max_depth': randint(5, 10)}

#agora cria um objeto searchCV e o ajusta aos dados
# now create a searchCV object and fit it to the data
search = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=0),
                            n_iter=5,
                            param_distributions=param_distributions,
                            random_state=0)
search.fit(X_train, y_train)


search.best_params_

#o objeto de pesquisa agora funciona como um estimador de floresta aleatória normal
#com max_depth=9 and n_estimators=4
search.score(X_test, y_test)


0.735363411343253

**Observação**<br>

Na prática, você quase sempre deseja pesquisar em um pipeline , em vez de um único estimador. Uma das principais razões é que, se você aplicar uma etapa de pré-processamento a todo o conjunto de dados sem usar um pipeline e, em seguida, realizar qualquer tipo de validação cruzada, estará quebrando a suposição fundamental de independência entre dados de treinamento e teste. De fato, como você pré-processou os dados usando todo o conjunto de dados, algumas informações sobre os conjuntos de teste estão disponíveis para os conjuntos de treinamento. Isso levará a superestimar o poder de generalização do estimador (você pode ler mais neste [post do Kaggle](https://www.kaggle.com/alexisbcook/data-leakage) ).
O uso de um pipeline para validação cruzada e pesquisa o manterá longe dessa armadilha comum.

## Próximos passos

Cobrimos brevemente o ajuste e a previsão de estimadores, etapas de pré-processamento, pipelines, ferramentas de validação cruzada e pesquisas automáticas de hiperparâmetros. Este guia deve fornecer uma visão geral de alguns dos principais recursos da biblioteca, mas há muito mais no ***scikit-learn***!

Consulte o [Guia do usuário do Scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)o para obter detalhes sobre todas as ferramentas que fornecemos. Você também pode encontrar uma lista completa da API pública na [API Reference](https://scikit-learn.org/stable/modules/classes.html#api-ref).

Você também pode ver nossos inúmeros [exemplos](https://scikit-learn.org/stable/auto_examples/index.html#general-examples) que ilustram o uso de ***scikit-learn*** em muitos contextos diferentes.

Os [tutoriais](https://scikit-learn.org/stable/tutorial/index.html#tutorial-menu) também contêm recursos de aprendizado adicionais.

## Referências
* https://scikit-learn.org/stable/getting_started.html
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier